In [1]:
# !pip install sentence_transformers

In [2]:
#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0.tar.gz

In [3]:
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
stopwords = list(STOP_WORDS)
nlp = spacy.load('en_core_web_sm')

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.cluster import KMeans

In [4]:
json1 = {}
json1['_id'] = 1
json1_text1 = {}
json1_text1['title'] = "Leah LaBelle"
json1_text1['text'] = "<p>Leah LaBelle Vladowski (September 8, 1986 – January 31, 2018) was an American singer\nShe rose to prominence in 2004 as a contestant on the third season of American Idol, placing twelfth in the season finals. In 2007, LaBelle began recording covers of R&B and soul music for her YouTube channel. These videos led to work as a backing vocalist starting in 2008 and a record deal in 2011 with Epic in partnership with I Am Other and So So Def Recordings. LaBelle released a sampler, three singles, and a posthumous extended play (EP).<br /></p>"
json1_text2 = {}
json1_text2['title'] = "The United Kingdom"
json1_text2['text'] = "<p>The United Kingdom of Great Britain and Northern Ireland, commonly known as the United Kingdom (UK) or Britain, is a country in Europe, off the north-western coast of the continental mainland. It comprises England, Scotland, Wales and Northern Ireland. The United Kingdom includes the island of Great Britain, the north-eastern part of the island of Ireland, and many smaller islands within the British Isles. Northern Ireland shares a land border with the Republic of Ireland; otherwise, the United Kingdom is surrounded by the Atlantic Ocean, the North Sea, the English Channel, the Celtic Sea and the Irish Sea. The total area of the United Kingdom is 242,495 square kilometres (93,628 sq mi), with an estimated 2020 population of more than 67 million people.<br /></p>"
json1['texts'] = [json1_text1, json1_text2]

json2 = {}
json2['_id'] = 2
json2_text1 = {}
json2_text1['title'] = "Leah LaBelle"
json2_text1['text'] = "<p>Born in Toronto, Ontario, and raised in Seattle, Washington, LaBelle began pursuing music ambition as a career in her teens. As a child, she performed in the Total Experience Gospel Choir and the musical Black Nativity. In 2005, LaBelle attended the Berklee College of Music for a year before dropping out and relocating to Los Angeles. While in college, she collaborated with Andreao Heard on a demo. Following the advice of an industry contact, LaBelle released her music through her YouTube channel. Keri Hilson hired LaBelle as a backing vocalist after hearing her rendition of \"Energy\", which led to her working for other artists on their tours.<br /></p>"
json2_text2 = {}
json2_text2['title'] = "The United Kingdom"
json2_text2['text'] = None
json2['texts'] = [json2_text1, json2_text2]

json3 = {}
json3['_id'] = 3
json3_text1 = {}
json3_text1['title'] = "Leah LaBelle"
json3_text1['text'] = "<p>LaBelle signed a record deal after Pharrell Williams and Jermaine Dupri contacted her. Her sampler Pharrell Williams and Jermaine Dupri Present Leah LaBelle (2012) was distributed only to record companies. It was supported by two singles, \"Sexify\" and \"What Do We Got to Lose?\" LaBelle received the Soul Train Centric Award at the 2012 Soul Train Music Awards. She released the non-album single \"Lolita\" the following year. On January 31, 2018, LaBelle and her boyfriend Rasual Butler died in a single vehicle car crash in Los Angeles, when Butler, under the influence of alcohol and drugs, lost control of the car. A posthumous EP, Love to the Moon, was released on September 7, 2018.<br /></p>"
json3_text2 = {}
json3_text2['title'] = "The United Kingdom"
json3_text2['text'] = "<p>The United Kingdom has evolved from a series of annexations, unions and separations of constituent countries over several hundred years. The Treaty of Union between the Kingdom of England (which included Wales, annexed in 1542) and the Kingdom of Scotland in 1707 formed the Kingdom of Great Britain. Its union in 1801 with the Kingdom of Ireland created the United Kingdom of Great Britain and Ireland. Most of Ireland seceded from the UK in 1922, leaving the present United Kingdom of Great Britain and Northern Ireland, which formally adopted that name in 1927. The nearby Isle of Man, Guernsey and Jersey are not part of the UK, being Crown Dependencies, but the British Government is responsible for their defence and international representation. There are also 14 British Overseas Territories, the last remnants of the British Empire which, at its height in the 1920s, encompassed almost a quarter of the world's landmass and a third of the world's population, and was the largest empire in history. British influence can be observed in the language, culture and the legal and political systems of many of its former colonies.<br /></p>"
json3['texts'] = [json3_text1, json3_text2]

fin_json = [json1, json2, json3]
fin_json

[{'_id': 1,
  'texts': [{'title': 'Leah LaBelle',
    'text': '<p>Leah LaBelle Vladowski (September 8, 1986 – January 31, 2018) was an American singer\nShe rose to prominence in 2004 as a contestant on the third season of American Idol, placing twelfth in the season finals. In 2007, LaBelle began recording covers of R&B and soul music for her YouTube channel. These videos led to work as a backing vocalist starting in 2008 and a record deal in 2011 with Epic in partnership with I Am Other and So So Def Recordings. LaBelle released a sampler, three singles, and a posthumous extended play (EP).<br /></p>'},
   {'title': 'The United Kingdom',
    'text': '<p>The United Kingdom of Great Britain and Northern Ireland, commonly known as the United Kingdom (UK) or Britain, is a country in Europe, off the north-western coast of the continental mainland. It comprises England, Scotland, Wales and Northern Ireland. The United Kingdom includes the island of Great Britain, the north-eastern part of t

In [5]:
with open('summary_json.json', mode='w') as f:
    json.dump(fin_json, f)

In [6]:
with open('summary_json.json') as f:
    js = json.load(f)
js

[{'_id': 1,
  'texts': [{'title': 'Leah LaBelle',
    'text': '<p>Leah LaBelle Vladowski (September 8, 1986 – January 31, 2018) was an American singer\nShe rose to prominence in 2004 as a contestant on the third season of American Idol, placing twelfth in the season finals. In 2007, LaBelle began recording covers of R&B and soul music for her YouTube channel. These videos led to work as a backing vocalist starting in 2008 and a record deal in 2011 with Epic in partnership with I Am Other and So So Def Recordings. LaBelle released a sampler, three singles, and a posthumous extended play (EP).<br /></p>'},
   {'title': 'The United Kingdom',
    'text': '<p>The United Kingdom of Great Britain and Northern Ireland, commonly known as the United Kingdom (UK) or Britain, is a country in Europe, off the north-western coast of the continental mainland. It comprises England, Scotland, Wales and Northern Ireland. The United Kingdom includes the island of Great Britain, the north-eastern part of t

In [7]:
# convert json to pandas DataFrame
df = pd.json_normalize(js)
df

,_id,texts
0,1,"[{'title': 'Leah LaBelle', 'text': '<p>Leah La..."
1,2,"[{'title': 'Leah LaBelle', 'text': '<p>Born in..."
2,3,"[{'title': 'Leah LaBelle', 'text': '<p>LaBelle..."


In [8]:
# split list of texts in each id into multiple records
df = df.explode('texts')
df

,_id,texts
0,1,"{'title': 'Leah LaBelle', 'text': '<p>Leah LaB..."
0,1,"{'title': 'The United Kingdom', 'text': '<p>Th..."
1,2,"{'title': 'Leah LaBelle', 'text': '<p>Born in ..."
1,2,"{'title': 'The United Kingdom', 'text': None}"
2,3,"{'title': 'Leah LaBelle', 'text': '<p>LaBelle ..."
2,3,"{'title': 'The United Kingdom', 'text': '<p>Th..."


In [9]:
# convert fields of texts dict to columns
df = pd.concat([df[['_id']], df['texts'].apply(pd.Series)], axis=1)
df

,_id,title,text
0,1,Leah LaBelle,"<p>Leah LaBelle Vladowski (September 8, 1986 –..."
0,1,The United Kingdom,<p>The United Kingdom of Great Britain and Nor...
1,2,Leah LaBelle,"<p>Born in Toronto, Ontario, and raised in Sea..."
1,2,The United Kingdom,None
2,3,Leah LaBelle,<p>LaBelle signed a record deal after Pharrell...
2,3,The United Kingdom,<p>The United Kingdom has evolved from a serie...


In [10]:
# extract text from html
df['text'] = df['text'].apply(lambda x: BeautifulSoup(x).get_text() if(pd.notnull(x)) else x)
df

,_id,title,text
0,1,Leah LaBelle,"Leah LaBelle Vladowski (September 8, 1986 – Ja..."
0,1,The United Kingdom,The United Kingdom of Great Britain and Northe...
1,2,Leah LaBelle,"Born in Toronto, Ontario, and raised in Seattl..."
1,2,The United Kingdom,None
2,3,Leah LaBelle,LaBelle signed a record deal after Pharrell Wi...
2,3,The United Kingdom,The United Kingdom has evolved from a series o...


In [11]:
# group texts of same title
df = df.loc[df['text'].notna(), :].groupby('title', as_index=False).agg({'text': '. '.join})
df

,title,text
0,Leah LaBelle,"Leah LaBelle Vladowski (September 8, 1986 – Ja..."
1,The United Kingdom,The United Kingdom of Great Britain and Northe...


In [12]:
def clean_text(txt):
    txt = re.sub('\n', '. ', txt)
    txt = re.sub('\.\s+(?=\.)', '.', txt)
    txt = re.sub('\.+', '.', txt)
    txt = re.sub('\s+', ' ', txt)
    return txt

df['text'] = df['text'].apply(clean_text)
df

,title,text
0,Leah LaBelle,"Leah LaBelle Vladowski (September 8, 1986 – Ja..."
1,The United Kingdom,The United Kingdom of Great Britain and Northe...


In [13]:
def count_summary(txt, n_top=5):
    doc = nlp(txt)
    tokens = [token.text for token in doc]
    
    word_frequencies = {}
    for word in doc:
        if word.text.lower() not in stopwords:
            if re.search('[a-zA-Z]+', word.text.lower()):
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    k = Counter(word_frequencies)
    
    max_frequency = max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word] = word_frequencies[word]/max_frequency
        
    sentence_tokens = [sent for sent in doc.sents]
    
    sentence_scores = {}
    for sent in sentence_tokens:
        i = 0
        for word in sent:
            i += 1
            if word.text in word_frequencies.keys():
                if sent.text not in sentence_scores.keys():
                    sentence_scores[sent.text] = word_frequencies[word.text]
                else:
                    sentence_scores[sent.text] += word_frequencies[word.text]
        if sent.text in sentence_scores.keys():
            sentence_scores[sent.text] = sentence_scores[sent.text]/i
            
    summarized_sentences = nlargest(n_top, sentence_scores, key=sentence_scores.get)
    final_sentences = [w for w in summarized_sentences]
    return " ".join(final_sentences)

def rank_summary(txt, n_top=5):
    doc = nlp(txt)
    sentence_tokens = [sent.text for sent in doc.sents]
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(sentence_tokens)
    sim_matrix = cosine_similarity(embeddings)
    
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)
    
    ranked_sentences = sorted(((scores[i], s) for i,s in enumerate(sentence_tokens)), reverse=True)
    
    summary = " ".join([i[1] for i in ranked_sentences[:n_top]])
    return summary

def cluster_summary(txt, n_top=5):
    doc = nlp(txt)
    sentence_tokens = [sent.text for sent in doc.sents]
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(sentence_tokens)
    
    kmeans = KMeans(n_clusters=n_top)
    kmeans.fit(embeddings)
    
    avg = []
    for j in range(n_top):
        idx = np.where(kmeans.labels_ == j)[0]
        avg.append(np.mean(idx))
    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, embeddings)
    ordering = sorted(range(n_top), key=lambda k: avg[k])
    summary = ' '.join([sentence_tokens[closest[idx]] for idx in ordering])
    return summary

In [14]:
df['count_summary'] = df['text'].apply(lambda x: count_summary(x, n_top=5))
df['rank_summary'] = df['text'].apply(lambda x: rank_summary(x, n_top=5))
df['cluster_summary'] = df['text'].apply(lambda x: cluster_summary(x, n_top=5))
df

,title,text,count_summary,rank_summary,cluster_summary
0,Leah LaBelle,"Leah LaBelle Vladowski (September 8, 1986 – Ja...",LaBelle signed a record deal after Pharrell Wi...,"In 2007, LaBelle began recording covers of R&B...",and These videos led to work as a backing voca...
1,The United Kingdom,The United Kingdom of Great Britain and Northe...,Its union in 1801 with the Kingdom of Ireland ...,The United Kingdom of Great Britain and Northe...,The United Kingdom includes the island of Grea...
